##### From Week 1

In [36]:
import warnings
warnings.filterwarnings('ignore')

import os
from os.path import join

import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import xgboost as xgb
import lightgbm as lgb

import seaborn as sns
import matplotlib.pyplot as plt

# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

train_data_path = join('./data/train.csv')
sub_data_path = join('./data/test.csv')

data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)

y = data['price']
del data['price']

train_len = len(data)
data = pd.concat((data, sub), axis=0)

sub_id = data['id'][train_len:]
del data['id']

data['date'] = data['date'].apply(lambda x : str(x[:6])).astype(int)

skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)

y_log_transformation = np.log1p(y)

sub = data.iloc[train_len:, :] 
x = data.iloc[:train_len, :] 

print(x.shape)
print(sub.shape)

(15035, 19)
(6468, 19)


##### Averaging

In [37]:
gboost = GradientBoostingRegressor(random_state=2023)
xgboost = xgb.XGBRegressor(random_state=2023)
lightgbm = lgb.LGBMRegressor(random_state=2023)

models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
          {'model':lightgbm, 'name':'LightGBM'}]

##### Cross Validation Function

In [38]:
def get_cv_score(models):
    kfold = KFold(n_splits=5).get_n_splits(x.values)
    for m in models:
        CV_score = np.mean(cross_val_score(m['model'], X=x.values, y=y, cv=kfold))
        print(f"Model: {m['name']}, CV score:{CV_score:.4f}")

In [39]:
get_cv_score(models)

Model: GradientBoosting, CV score:0.8609
Model: XGBoost, CV score:0.8861
Model: LightGBM, CV score:0.8819


##### Ensemble (Averaging Blending)

In [40]:
def AveragingBlending(models, x, y, sub_x):
    # Use x and y to train models
    # x : Pandas DataFrame object
    # x.values : NumPy array form
    for m in models : 
        m['model'].fit(x.values, y)
    
    # Model prediction using sub_x
    # Store the prediction results into predictions
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])

    # Return mean of each model
    return np.mean(predictions, axis=1)

# print(models)
# sub.head()
# x.head()
# print(x.values)

In [41]:
y_pred = AveragingBlending(models, x, y, sub)
print(len(y_pred))
y_pred

6468


array([ 529966.66304912,  430726.21272617, 1361676.91242777, ...,
        452081.69137012,  341572.97685942,  421725.1231835 ])

In [42]:
result = pd.DataFrame({
    'id' : sub_id,
    'price' : y_pred
})

result.head()

,id,price
0,15035,5.299667e+05
1,15036,4.307262e+05
2,15037,1.361677e+06
3,15038,3.338036e+05
4,15039,3.089006e+05


In [43]:
submission_path = './data/submission.csv'

result.to_csv(submission_path, index=False)

##### Better Model

In [44]:
train = x
test = sub

# Need to use expm1 after model prediction
y = np.log1p(y)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [46]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

In [47]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [48]:
# random_state = None
random_state = 2023

gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

In [49]:
def get_scores(models, train, y):
    df = {} # create table for saving scores
    for model in models:
        model_name = model.__class__.__name__
        # validation : 0.2
        X_train, X_val, y_train, y_val = train_test_split(train, y, random_state=random_state, test_size=0.2)

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)

        df[model_name] = rmse(y_val, y_pred)

        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
    
    return score_df

##### Grid Search / Random Search
Hyperparameter tuning in machine learning models
##### Grid Search : predefined set of hyperparameter values is specified
##### Random Search : randomly selects hyperparameter values from a predefined search space

In [50]:
# Grid Search
from sklearn.model_selection import GridSearchCV

In [51]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [1, 10],
}

In [52]:
model = LGBMRegressor(random_state=random_state)

In [53]:
grid_model = GridSearchCV(model, param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5, verbose=1, n_jobs=5)

grid_model.fit(x,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=2023), n_jobs=5,
             param_grid={'max_depth': [1, 10], 'n_estimators': [50, 100]},
             scoring='neg_mean_squared_error', verbose=1)